In [6]:
from sklearn.feature_selection import SelectFdr, f_classif
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import wilcoxon
from scipy.stats import ranksums
from statsmodels.stats.multitest import fdrcorrection
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif,f_regression
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc, mean_squared_error, precision_score, jaccard_score, fowlkes_mallows_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from boruta import BorutaPy
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

In [10]:
rna = pd.read_csv('D:/multiomics/2025june/rna_rb.csv', sep=',')
lipid = pd.read_csv('D:/multiomics/2025june/lipid_rb.csv', sep=',')
meta = pd.read_csv('D:/multiomics/2025june/meta_rb.csv', sep=',')

In [11]:
sets_to_intersect = [
    set(rna['id']),
    set(lipid['id']),
    set(meta['id'])
]

common_ids = set.intersection(*sets_to_intersect)

sorted_common_ids = sorted(common_ids)

meta = meta[meta['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()
lipid = lipid[lipid['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()
rna = rna[rna['id'].isin(common_ids)].set_index('id').loc[sorted_common_ids].reset_index()

In [12]:
sum(rna['id']==meta['id']),sum(rna['id']==lipid['id'])

(156, 156)

In [13]:
f_rna = rna.iloc[:,11:]
f_lipid = lipid.iloc[:,6:]
f_meta = meta.iloc[:,6:]
f_agender = rna.iloc[:,1:3]

In [14]:
du = (rna['sVNT_M1_WT_RBD_10_times_dilution']-rna['sVNT_M6_WT_RBD_10_times_dilution'])/rna['sVNT_M1_WT_RBD_10_times_dilution']

In [16]:
label = (du<=du.quantile(0.25))*1

In [17]:
batch_label = pd.concat([rna[['id','Age','Gender','Batch']], label], axis=1) 
batch_label = batch_label.rename(columns={0: 'label'})

In [18]:
batch_label['batch_label'] = (
    batch_label['Batch'].astype(str).str.cat(batch_label['label'].astype(str), sep='_')
)

In [19]:
cross_table = pd.crosstab(batch_label['label'], batch_label['Batch'])

In [20]:
cross_table

Batch,Batch01,Batch02
label,,
0,51,66
1,16,23


In [21]:
combine = pd.concat([f_rna, f_lipid, f_meta], axis=1) 
combine = pd.concat([f_agender, combine], axis=1)

In [22]:
X_train, X_test, y_train, y_test = train_test_split(combine, label, test_size=0.2,stratify=batch_label['batch_label'],random_state=42)

In [36]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestClassifier
from functools import partial
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import f_classif
from joblib import Parallel, delayed
import pandas as pd

def process_fold(train_idx, val_idx, i, X_train, y_train):
    print(f"Processing fold {i}")
    
    X_fold_train, y_fold_train = X_train.iloc[train_idx], y_train.iloc[train_idx]
    
    F_values, p_values = f_classif(X_fold_train, y_fold_train)
    significant_features_mask = p_values < 0.05
    significant_features = X_fold_train.columns[significant_features_mask]
    print(f"Fold {i+1}: {len(significant_features)} significant features")
    X_fold_train_sig = X_fold_train[significant_features]

    estimator = RandomForestClassifier(random_state=42, n_jobs=-1)
    
    boruta_selector = BorutaPy(
        estimator=estimator,
        n_estimators='auto',  
        verbose=0,           
        random_state=42,
        max_iter=100,       
        perc=90,alpha=0.01            
    )
    
    boruta_selector.fit(X_fold_train_sig.values, y_fold_train.values)
    
    selected_features_mask = boruta_selector.support_
    selected_features = significant_features[selected_features_mask]
    
    estimator.fit(X_fold_train_sig[selected_features], y_fold_train)
    importances = estimator.feature_importances_

    return [
        {
            'fold': i,
            'feature': feature,
            'importance': importance,
        }
        for feature, importance in zip(selected_features, importances)
    ]

def main():

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    
    results = Parallel(n_jobs=-1, verbose=10)(
        delayed(partial(process_fold, X_train=X_train, y_train=y_train))(train_idx, val_idx, i)
        for i, (train_idx, val_idx) in enumerate(cv.split(X_train, batch_label['batch_label'][X_train.index]))
    )
    
    results = [item for sublist in results for item in sublist]
    
    results_df = pd.DataFrame(results)
    
    results_df = results_df.sort_values(['fold', 'importance'], ascending=[True, False])
    
    print(results_df.head())
    return results_df

if __name__ == '__main__':
    all_feature_counts = main()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:  2.3min remaining:  5.4min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:  2.3min remaining:  2.3min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  2.4min remaining:  1.0min


     fold                                       feature  importance
119     0      ENSG00000217930.8__PAM16__protein.coding    0.021552
82      0  ENSG00000172922.11__RNASEH2C__protein.coding    0.014634
162     0                     ENSG00000267547.1__NA__NA    0.014561
158     0                     ENSG00000261396.1__NA__NA    0.012551
12      0     ENSG00000101849.18__TBL1X__protein.coding    0.011427


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.5min finished


In [37]:
feature_counts = all_feature_counts['feature'].value_counts()
common_features = feature_counts[feature_counts >= 7].index
df_filtered = all_feature_counts[all_feature_counts['feature'].isin(common_features)]

stats = df_filtered.groupby('feature')['importance'].agg(['sum', 'mean', 'count'])
stats = stats.sort_values('count', ascending=False)

top_feature = stats.index[:3]
feature_selected = list(top_feature)

In [38]:
feature_selected

['ENSG00000217930.8__PAM16__protein.coding',
 'ENSG00000134245.18__WNT2B__protein.coding',
 'PC.38.6e..rep..rep..rep.']

In [44]:
# Prepare training and test sets
Xtrain = X_train[['Age', 'Gender'] + feature_selected]
Xtest = X_test[['Age', 'Gender'] + feature_selected]

# Initialize and train Random Forest model
rf = RandomForestClassifier(random_state=42)
rf.fit(Xtrain, y_train)

# Predict probabilities for the positive class
y_pred_proba = rf.predict_proba(Xtest)[:, 1]

# Calculate AUC
auc = roc_auc_score(y_test, y_pred_proba)

print(f'{auc:.2f}')

0.65
